In [1]:
%%configure
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
24,application_1583130900432_0025,pyspark,idle,Link,Link,


In [2]:
//NOTE: BEFORE RUNNING ABOVE CONFIGURE STATEMENT, ENSURE THAT HUDI JARS ARE LOCATED WITHIN HDFS. 
//SEE README.MD FOR DIRECTIONS.

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
26,application_1583130900432_0026,spark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.DataSourceReadOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor


In [4]:
//Hudi Table을 저장하기 위해 앞에서 생성한 S3 버킷을 지정해줍니다. Where to store your Hudi Table.
val outputBucket = "analytics-seung-hudi-seoul"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

outputBucket: String = analytics-seung-hudi-seoul


In [5]:
//샘플 데이터로 Amazon 상품 리뷰 데이터를 사용합니다. 
val inputDatasetBucket = "amazon-reviews-pds"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

inputDatasetBucket: String = amazon-reviews-pds


In [6]:
// Hudi option에 들어갈 기본 설정값을 입력합니다. 
//Specify common DataSourceWriteOptions in the single hudiOptions variable 
val hudiTableName = "amazon_product_reviews"
val hudiTableRecordKey = "review_id"
val hudiTablePath = "s3://" + outputBucket + "/createdatasets/" + hudiTableName
val hudiTablePartitionColumn = "review_date"
val hudiTablePrecombineKey = "timestamp"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiTableName: String = amazon_product_reviews
hudiTableRecordKey: String = review_id
hudiTablePath: String = s3://analytics-seung-hudi-seoul/createdatasets/amazon_product_reviews
hudiTablePartitionColumn: String = review_date
hudiTablePrecombineKey: String = timestamp


In [7]:
/****************************
Read out Amazon product reviews table
*****************************/
val sourceData = (spark.read.parquet("s3://" + inputDatasetBucket + "/parquet/product_category=Home_Improvement/*")
                            .withColumn(hudiTablePrecombineKey, current_timestamp().cast("long"))
                            .withColumn(hudiTablePartitionColumn, regexp_replace(col(hudiTablePartitionColumn), "-", "/"))
                            .cache())

sourceData.select("marketplace", "review_id", "customer_id", "product_title", 
                  "star_rating", "review_date").show(10);

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sourceData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [marketplace: string, customer_id: string ... 14 more fields]
+-----------+--------------+-----------+--------------------+-----------+-----------+
|marketplace|     review_id|customer_id|       product_title|star_rating|review_date|
+-----------+--------------+-----------+--------------------+-----------+-----------+
|         US|R3MK25NME5LTNQ|   41916184|Decorative Switch...|          5| 2014/11/17|
|         US|R10P620FA77MVJ|   33516140|Delta 75152 2.5 G...|          5| 2009/05/12|
|         US| RKVCGQRPHDNZ8|   17486082|Neroli 100% Pure ...|          5| 2014/11/17|
|         US|R2KQ8CFO5FBW4F|   17359526|Traditional / Cla...|          5| 2009/05/12|
|         US|R2NKA708V5GPGI|   15361253|Ibis & Orchid Dra...|          2| 2014/11/17|
|         US|R2BNIIP9LQK2RG|   43316183|Lux WIN100 Heatin...|          5| 2009/05/12|
|         US|R163IKXC5VYC50|   33660121|     MLN-55VARIATION|          5| 2014/11/17|
|        

In [8]:
/****************************
다양한 Hudi 옵션을 지정합니다. 데이터 저장 타입과, 파티션키, 리오더키 등
Our Hudi Options for our Product Reviews Dataset.
*****************************/
val hudiOptions = Map[String,String](
  HoodieWriteConfig.TABLE_NAME -> hudiTableName,

  //For this data set, we will configure it to use the COPY_ON_WRITE storage strategy. 
  //You can also choose MERGE_ON_READ
  DataSourceWriteOptions.STORAGE_TYPE_OPT_KEY -> "COPY_ON_WRITE", 

  //These three options configure what Hudi should use as its record key, 
  //partition column, and precombine key.
  DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "review_id",
  DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "timestamp",
  DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "review_date",

  //For this data set, we specify that we want to sync metadata with Hive. 
  DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true",
  DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName,
  DataSourceWriteOptions.HIVE_ASSUME_DATE_PARTITION_OPT_KEY -> "false",
  DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "year,month,day",
  DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY ->
                                        classOf[MultiPartKeysValueExtractor].getName
)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> timestamp, hoodie.datasource.hive_sync.partition_fields -> year,month,day, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> amazon_product_reviews, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> review_id, hoodie.table.name -> amazon_product_reviews, hoodie.datasource.write.storage.type -> COPY_ON_WRITE, hoodie.datasource.hive_sync.assume_date_partitioning -> false, hoodie.datasource.write.partitionpath.field -> review_date)


In [9]:
/** ***************************************************************
앞에서 읽은 샘플 데이터를 Hudi 옵션에 맞춰서 S3에 Hudi Dataset 형식으로 저장합니다. 
비교적 시간이 걸리는 작업입니다. 
Lets write our input dataset to Hudi.
*******************************************************************/
(sourceData.write
  .format("org.apache.hudi")
  .options(hudiOptions)

  //Operation Key tells Hudi whether this is an Insert, Upsert, or Bulk Insert operation
  .option(DataSourceWriteOptions.OPERATION_OPT_KEY, 
                                   DataSourceWriteOptions.BULK_INSERT_OPERATION_OPT_VAL)
  
  .mode(SaveMode.Overwrite)
  .save(hudiTablePath))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org.apache.hudi.hive.HoodieHiveSyncException: Cannot create hive connection jdbc:hive2://localhost:10000/
  at org.apache.hudi.hive.HoodieHiveClient.createHiveConnection(HoodieHiveClient.java:549)
  at org.apache.hudi.hive.HoodieHiveClient.<init>(HoodieHiveClient.java:108)
  at org.apache.hudi.hive.HiveSyncTool.<init>(HiveSyncTool.java:60)
  at org.apache.hudi.HoodieSparkSqlWriter$.syncHive(HoodieSparkSqlWriter.scala:235)
  at org.apache.hudi.HoodieSparkSqlWriter$.write(HoodieSparkSqlWriter.scala:169)
  at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala:91)
  at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
 

In [10]:
/** **********************************
Querying Hudi data is easy. We set the format to "org.apache.hudi"
**************************************/
val readOptimizedHudiViewDF = (spark.read
       .format("org.apache.hudi")
       .load(hudiTablePath + "/*/*/*/*")
       .cache())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimizedHudiViewDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 19 more fields]


In [11]:
/** ***********************************
Lets take a look at our data. Lets say someone says there is something odd going 
on with star ratings.
**************************************/
readOptimizedHudiViewDF.registerTempTable("amazon_product_reviews_raw_ro_table");
spark.sql("""select star_rating, count(*) from amazon_product_reviews_raw_ro_table 
                                group by star_rating order by star_rating ASC""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+
|star_rating|count(1)|
+-----------+--------+
|          1|  246630|
|          2|  124792|
|          3|  189490|
|          4|  419488|
|          5| 1660254|
+-----------+--------+



In [12]:
/** *********************************
Select the rows we want to update and and make the update.
************************************/
val upsertdf = (readOptimizedHudiViewDF.filter($"star_rating" === 100)
                                .withColumn("star_rating", lit(null).cast(IntegerType)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

upsertdf: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 19 more fields]


In [13]:
/** ********************************
Before, if you wanted to update data in S3, you had to read the old data, merge with the new data, and then overwrite
the old data. Now, with Hudi, you can directly update the data in-place.
************************************/
(upsertdf.write
  .format("org.apache.hudi")
  .options(hudiOptions)
  .option(DataSourceWriteOptions.OPERATION_OPT_KEY, 
                    DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
  .mode(SaveMode.Append)
  .save(hudiTablePath))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
val readOptimizedHudiViewDF = (spark.read.format("org.apache.hudi")
                                    .load(hudiTablePath + "/*/*/*/*").cache())
readOptimizedHudiViewDF.registerTempTable("amazon_product_reviews_ro_table");

spark.sql("""select star_rating, count(*) from amazon_product_reviews_ro_table 
                        group by star_rating order by star_rating ASC""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimizedHudiViewDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 19 more fields]
+-----------+--------+
|star_rating|count(1)|
+-----------+--------+
|          1|  246630|
|          2|  124792|
|          3|  189490|
|          4|  419488|
|          5| 1660254|
+-----------+--------+



In [15]:
/** *******************************
Now, suppose we need to delete a customers information due to GDPR because a 
request was made by a customer?
***********************************/
val deleteRowsDf = readOptimizedHudiViewDF.filter($"customer_id" === 32068341);

//Deletion
(deleteRowsDf.write
  .format("org.apache.hudi")
  .options(hudiOptions)
  //We set the operation to UPSERT
  .option(DataSourceWriteOptions.OPERATION_OPT_KEY, 
                       DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL)
  //We set the Payload Class to be empty record 
  .option(DataSourceWriteOptions.PAYLOAD_CLASS_OPT_KEY, 
                       "org.apache.hudi.EmptyHoodieRecordPayload")
  .mode(SaveMode.Append)
  .save(hudiTablePath))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

org.apache.hudi.hive.HoodieHiveSyncException: Cannot create hive connection jdbc:hive2://localhost:10000/
  at org.apache.hudi.hive.HoodieHiveClient.createHiveConnection(HoodieHiveClient.java:549)
  at org.apache.hudi.hive.HoodieHiveClient.<init>(HoodieHiveClient.java:108)
  at org.apache.hudi.hive.HiveSyncTool.<init>(HiveSyncTool.java:60)
  at org.apache.hudi.HoodieSparkSqlWriter$.syncHive(HoodieSparkSqlWriter.scala:235)
  at org.apache.hudi.HoodieSparkSqlWriter$.write(HoodieSparkSqlWriter.scala:169)
  at org.apache.hudi.DefaultSource.createRelation(DefaultSource.scala:91)
  at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:45)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:70)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:68)
  at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:86)
 

In [16]:
val readOptimizedHudiViewDF = (spark.read.format("org.apache.hudi")
                               .load(hudiTablePath + "/*/*/*/*").cache())
readOptimizedHudiViewDF.registerTempTable("amazon_product_reviews_ro_table");

spark.sql("""select count(*) from amazon_product_reviews_ro_table where
                                    customer_id = 32068341""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

readOptimizedHudiViewDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 19 more fields]
+--------+
|count(1)|
+--------+
|       0|
+--------+



In [17]:
/**************************************
We can also do point in time queries. Lets take a look at all the commits.
***************************************/
val commits = (spark.sql("""select distinct(_hoodie_commit_time) as commitTime from 
                            amazon_product_reviews_ro_table order by commitTime""")
                .map(k => k.getString(0)).take(50))
commits.toString()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

commits: Array[String] = Array(20200318090304)
res32: String = [Ljava.lang.String;@4679d7e5


In [18]:
/** ********************************
Suppose that we wanted to know what was a review at a certain point of time or a range
of commits.
************************************/
val beginTime = "0"
val endTime = commits(0) // commit time we are interested in

val amazon_product_reviews_table = (spark.read
     .format("org.apache.hudi")
     //Mark that we want to do an incremental query
     .option(DataSourceReadOptions.VIEW_TYPE_OPT_KEY, 
                             DataSourceReadOptions.VIEW_TYPE_INCREMENTAL_OPT_VAL)

     //Set at what time we want to start quering.
     .option(DataSourceReadOptions.BEGIN_INSTANTTIME_OPT_KEY, beginTime)
     .option(DataSourceReadOptions.END_INSTANTTIME_OPT_KEY, endTime)

     .options(hudiOptions)
     .load(hudiTablePath)).cache()

(amazon_product_reviews_table.select("review_id", "product_id", "product_title", 
                                   "star_rating").filter($"star_rating" === 100).show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

beginTime: String = 0
endTime: String = 20200318090304
amazon_product_reviews_table: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 19 more fields]
+---------+----------+-------------+-----------+
|review_id|product_id|product_title|star_rating|
+---------+----------+-------------+-----------+
+---------+----------+-------------+-----------+



In [19]:
/***********************************
Hive and Presto can query the data too!
************************************/

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…